In [ ]:
cd ~/SageMaker/lunar_climber

In [15]:
%%sh

# Specify an algorithm name
algorithm_name=lunar-climber

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/8 : FROM python:3.6-slim
 ---> 7710de4f3716
Step 2/8 : WORKDIR /opt/ml/code/
 ---> Using cache
 ---> d6e582bc54a4
Step 3/8 : COPY . .
 ---> 4aeee8d38996
Step 4/8 : RUN apt-get update &&     apt-get install --no-install-recommends -y     gcc swig python3-dev libgl1-mesa-glx libglib2.0-0 &&     apt-get clean && rm -rf /var/lib/apt/lists/*
 ---> Running in fadf524626c0
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [224 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7906 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7868 B]
Fetched 8378 kB in 2s (4301 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The fol

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [16]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'lunar-climber'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'

byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

byoc_image_uri
# This should return something like
# 111122223333.dkr.ecr.us-east-2.amazonaws.com/sagemaker-byoc-test:latest

'733927475662.dkr.ecr.us-east-2.amazonaws.com/lunar-climber:latest'

In [18]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

estimator = Estimator(image_name=byoc_image_uri,
                      role=get_execution_role(),
                      base_job_name='lunar-climber-test-job',
                      train_instance_count=1,
                      train_instance_type='local')

# start training
estimator.fit()

# deploy the trained model
# predictor = estimator.deploy(1, instance_type)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


Creating tmpuygz4hst_algo-1-lj0v9_1 ... 
Attaching to tmpuygz4hst_algo-1-lj0v9_12mdone
algo-1-lj0v9_1  | 2020-09-04 06:11:45,292 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
algo-1-lj0v9_1  | /usr/local/bin/python -m pip install -r requirements.txt
algo-1-lj0v9_1  | Requirement already satisfied: gym in /usr/local/lib/python3.6/site-packages (from -r requirements.txt (line 1)) (0.17.2)
algo-1-lj0v9_1  | Requirement already satisfied: stable-baselines in /usr/local/lib/python3.6/site-packages (from -r requirements.txt (line 2)) (2.10.1)
algo-1-lj0v9_1  | Requirement already satisfied: tensorflow==1.15.0 in /usr/local/lib/python3.6/site-packages (from -r requirements.txt (line 3)) (1.15.0)
algo-1-lj0v9_1  | Requirement already satisfied: Pyperclip in /usr/local/lib/python3.6/site-packages (from -r requirements.txt (line 6)) (1.8.0)
algo-1-lj0v9_1  | Requirement already satisfied: Box2D in /usr/local/lib/python3.6/site-packages (from -r requirements.t